In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [6]:
!ls /datasets/quora

sample_submission.csv  test.csv  train.csv


In [3]:
# Load training and test set
train_df = pd.read_csv('/datasets/quora/train.csv')
test_df = pd.read_csv('/datasets/quora/test.csv')

stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
question1 = []
for x in train_df["question1"]:
    question1.append(text_to_word_list(x))
question2 = []
for x in train_df["question2"]:
    question2.append(text_to_word_list(x))
question = question2 + question1

In [25]:
import gensim
from gensim.models import Word2Vec

In [26]:
word2vec = Word2Vec(question, size =300, window = 3, min_count=1,sg = 1)
print(word2vec)

Word2Vec(vocab=86005, size=300, alpha=0.025)


In [27]:
word2vec.save('word2vec.bin')

In [5]:
from gensim.models import word2vec
word2vec = word2vec.Word2Vec.load('word2vec.bin')

In [6]:
print(word2vec)

Word2Vec(vocab=86005, size=300, alpha=0.025)


In [8]:
# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']

questions_cols = ['question1', 'question2']

for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        for question in questions_cols:

            q2n = []
            for word in text_to_word_list(row[question]):

                if word in stops and word not in word2vec.wv.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            dataset.set_value(index, question, q2n)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [15]:
train_df.to_csv (r'trainE.csv', index = False)
test_df.to_csv (r'testE.csv', index = False)

In [18]:
import json
with open('vocabulary.json', 'w') as fp:
    json.dump(vocabulary, fp)

In [ ]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)
embeddings[0] = 0 

In [11]:
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.wv.vocab:
        embeddings[index] = word2vec[word]

del word2vec

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
